# HACETTEPE UNIVERSITY

Machine Learning Laboratory Assignment1

Name:FURKAN YILMAZ

Number:21527621

Subject:Movie Recommendation System

### INTRODUCTION

In the first part of this assignment, we are asked to solve the theoretical questions given to understand kNN and kernel regression algorithms.

In the second part, we are asked to recommend a movie to the users by using the kNN and weighted kNN algorithm, and to predict using the rating of the neighbors.

###  Part I: Theory Questions

### k-Nearest Neighbor Classification

1. The K nn algorithm is easy to implement but works quite slowly on large data sets. In large data sets, the number of variables is too high and the number of variables is too high causing the Knn algorithm to run slower.

2. a b a b a b a b  ------->     is a 1-Dimensional dataset.

When we select any point in this dataset for testing, we always take the error value as 1 because the closest point is not from its own class.




3. 	1) Since the Euclidean distance is used in the Knn algorithm, the test instance is assigned to (-) class since k = 1 is the closest point to that test (-).
	
2) When the euclidean distance is used for k = 3, the test sample is assigned to (-) since 2 of the nearest 3 points are (-) and 1 is (+).
	
3) When euclidean distance is used for k = 5, the test sample is assigned to (+) class since 3 of the nearest 5 points are (+) and 2 of them are (-).

4. 	

1) T

2) T

3) T

###  Linear Regression

1. The mean of x2 is 5.489,4 and the range is 8464 - 2025 = 6439.
x2^4 =  (4489 - 5489) / 6439 = - 0,155.

2. We always consider residual as vertical offsets. Perpendicular offset are useful in case of PCA.

3.<img src = "20191103_165735.jpg" >
    
4. Some data sets may show different characteristics in terms of size, variety  of variables, and range. In these cases, some features may be irrelevant or misleading. Feature scaling becomes meaningful in these cases. For example, if one  data in  the data set is larger, algorithms using euclidean distance need to normalize that data. 

### CONCLUSION

The error values I received when I applied kNN and weigtedKNN came very close to each other. I think it's because I didn't normalize the data. I found these values close to each other because I didn't normalize the data. I use different k numbers and got different results. Choosing k too small or too large gives incorrect results. While I set k to 7, 9 or 11, I usually find fewer errors. My error values are usually between 0.9 and 0.7, which may be due to the fact that the data is not cleared of bad data.



### SAMPLE OUTPUT

 
 k=11 and in this case the best model is 6th.
 
     1. Model Mean Absolute Error:0.8103398534607292
     1. Model weighted Error: 0.8094063876896627

     2. Model Mean Absolute Error:0.7739177080268705
     2. Model weighted Error: 0.7752481005699442

     3. Model Mean Absolute Error:0.7947319985162703
     3. Model weighted Error: 0.7949591410305946

     4. Model Mean Absolute Error:0.7894283405645401
     4. Model weighted Error: 0.7910210770538475

     5. Model Mean Absolute Error:0.8271934461187943
     5. Model weighted Error: 0.8294641036465963

     6. Model Mean Absolute Error:0.7404685713445363
     6. Model weighted Error: 0.7406271894095711

     7. Model Mean Absolute Error:0.8212376482597372
     7. Model weighted Error: 0.824360582139224

     8. Model Mean Absolute Error:0.8706121020418575
     8. Model weighted Error: 0.8706160012100945

     9. Model Mean Absolute Error:0.8122244949560911
     9. Model weighted Error: 0.811873015825695

     10. Model Mean Absolute Error:0.764928386462959
     10. Model weighted Error: 0.7667835308248643


In [431]:
import pandas as pd
import numpy as np
import math

I have read the supplied input files with the read_csv function of the pandas library.

In [432]:
readLinks=pd.read_csv("links.csv")
readMovies=pd.read_csv("movies.csv")
readRatings=pd.read_csv("ratings_train.csv")
readTags=pd.read_csv("tags.csv")
valid = pd.read_csv('ratings_test_without_gt_new.csv')


I combined the merge function with ratings dataframe and the movies dataframe. I created another dataframe that deleted the title genres and timestamp columns, and then sorted this dataframe by movieId column for future use

In [433]:
df1= readMovies.merge(readRatings,how='inner')
df2=df1.sort_values(by=['userId'])


df4=df2.drop(df1.columns[[1,2,5]],axis=1)
df4=df4.sort_values(by=['movieId'])

This function is used to calculate the cosine similarity of the two input vectors.

In [434]:
def cosineBasedSimilarty(x,y):
    dot = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    cosineSim = dot / (norm_x * norm_y)
    return cosineSim

The Knn function calculates the cosine similarity of each user in the test separately with each user in the train file.It uses the cosineBasedSimilarity function when calculating this.

In [435]:
def kNN(testArray,trainArray):
    similarityArray=[]
    for i in range(len(testArray)):
        newArray=[]
        for j in range(len(trainArray)):
            x= cosineBasedSimilarty(testArray[i],trainArray[j])
            newArray.append(x)
        similarityArray.append(newArray)
    userBySimilarty=np.array(similarityArray)
    return userBySimilarty


The neighbor function uses the table with cosine similarities to find the number of neighbors given the number of k and adds them to a dictionary.


In [436]:

def neighbor(k,newTable):
    dictt={}
    neighborsCosineDict={}
    for i in range(len(newTable)):
        similarityArray=[]
        neighborArray = []
        keys = newTable.index[i]
        for j in range(k):
            x = newTable.loc[newTable.index[i]].sort_values(ascending=False)
            similarityArray.append(x.values[j])
            neighborArray.append(x.index[j])
        neighborsCosineDict[keys]=similarityArray
        dictt[keys]=neighborArray
    return dictt,neighborsCosineDict
        


The meanAbsoluteError function uses the dataframe that holds the ratings given by its neighbors to the movies each test user  watches.Using this dataframe, predicts the rating that the user should give to the film, taking the average of neighbors.It then finds the difference between the test user's vote and the estimated value, and does this for all movies and calculates the user's error rate.

In [437]:
def meanAbsoluteError(table,neigs2):
    columndict2={}
    testdict2={}
    for i in range(len(table.columns)):
        a=table.columns[i]
        array=table[a].values
        counter=0
        avg=0
        for j in range(len(array)):
            if(array[j]!=0):
                counter=counter+1
                avg=avg+array[j]
        if(counter!=0):
            avg = avg/counter
            columndict2[a]=avg
            testdict2[a]= neigs2[i]
    errorByMovie=0
    for j in range(len(testdict2)): 
        errorByMovie += abs(list(testdict2.values())[j] - list(columndict2.values())[j])
    errorByMovie = errorByMovie/ len(testdict2)
    return errorByMovie





The weightedError function is similar to the meanAbsoluteError function. The only difference is multiplied by the cosine similarities of the ratings given and divided by the sum of the cosine similarities of the ratings.

In [438]:
def weightedError(simArray,table,neigs2):
    columndict={}
    testdict={}
    for i in range(len(table.columns)):
        a=table.columns[i]
        array=table[a].values
        sumOfArray=0
        avg=0
        for j in range(len(array)):
            if(array[j]!=0):
                sumOfArray += simArray[j]
                avg=avg+(array[j] * simArray[j])
        if(sumOfArray!=0):
            avg = avg/sumOfArray
            columndict[a]=avg
            testdict[a]= neigs2[i]

    errorByMovie2 = 0
    for j in range(len(testdict)): 
        errorByMovie2 += abs(list(testdict.values())[j] - list(columndict.values())[j])
    errorByMovie2 = errorByMovie2 / len(testdict)
    return errorByMovie2


The calculateError function creates variables to send to the meanAbsoluteError and weigtedError functions and passes these functions as parameters. Calculates the overall error rate after finding the error values of each test user.

In [439]:

def calculateError(testArray,trainArray,dictt,neighborsCosineDict,i):  
    TotalError=0
    weigtedKnnError=0
    for j in range(len(testArray)):
        df3=pd.DataFrame.from_dict(dictt,orient='index')
        b=list(dictt.keys())[j]
        a=df3.iloc[j].values

        real_index =[dataTable_indices[i] for i in a]
        indices = dataTable2.index == np.repeat(b, len(dataTable2))

        aa = dataTable[indices]
        bb = aa.values !=0
        control = df4['userId'] == list(dictt.keys())[j]
        c=df4[control]['movieId'].values
        
        neigs = dataTable.values[real_index]
        neigs = neigs[:, bb.tolist()[0]]
        table=pd.DataFrame(neigs,index=a,columns=c)
        test_index=dataTable_indices[b]
        test_index_rates=dataTable[indices]
        testMoviesRates = test_index_rates.values != 0

        neigs2 = dataTable.values[test_index]
        neigs2 = neigs2[testMoviesRates.tolist()[0]]

        tmp = meanAbsoluteError(table,neigs2)
        TotalError = TotalError + tmp

        simArray=list(neighborsCosineDict.values())[j]
        
        tmp2 = weightedError(simArray,table,neigs2)
        
        weigtedKnnError = weigtedKnnError + tmp2
        
    TotalError = TotalError/ (len(testArray))
    weigtedKnnError = weigtedKnnError / (len(testArray))
    print(" {}. Model Mean Absolute Error:{}".format(i,TotalError))    
    print(" {}. Model weighted Error: {}".format(i,weigtedKnnError))
    print()



In this section have been created pivot tables to be used.The values of the dataTable variable NaN are replaced with 0.This is done to calculate cosine similarity. The original file is shuffeled to determine the train file and the test file. A dictionary is created which specifies which index holds which userId for future use. I created a new dataFrame with the return value from knn function. the rows of this Dataframe are the users in the test, the columns are the trains, and the values of the cosine similarities.I did cross validation and called the calculateError function for each model and tried to see the most appropriate model by pressing the general error.

In [440]:
dataTable = pd.pivot_table(df1,values="rating",index='userId',columns='movieId')
dataTable2 = pd.pivot_table(df1,values="rating",index='userId',columns='movieId')
dataTable.fillna(value=0, inplace=True)
dataTable_indices = {}

for counter, i in enumerate(dataTable2.index):
    dataTable_indices[i] = counter

shuffleTable2=dataTable.iloc[np.random.permutation(len(dataTable))]

index=int((len(shuffleTable2))/10)
count=0

for i in range(10):
    if(i!=0):
        if(i==10):
            trainTable=shuffleTable2.iloc[0:count , :]
            testTable = shuffleTable2.iloc[count : , : ]
        else:
            train=shuffleTable2.iloc[0:count,:]
            
            testTable, trainTable = (shuffleTable2.iloc[count : index, : ], shuffleTable2.iloc[index : , : ])
            trainTable=pd.concat([train,trainTable])            
    else:      
        testTable, trainTable = (shuffleTable2.iloc[count : index, : ], shuffleTable2.iloc[index : , : ])

    index+=int((len(shuffleTable2))/10)
    count+=int((len(shuffleTable2))/10)
    testArray=np.array(testTable)
    trainArray = np.array(trainTable)
    testIndex=testTable.index

    similarity = kNN(testArray,trainArray)

    newTable = pd.DataFrame(similarity,index=testTable.index)
    newTable.columns=trainTable.index
    
    a,b = neighbor(11,newTable)

    
    calculateError(testArray,trainArray,a,b,i+1)

 1. Model Mean Absolute Error:0.7944159121939757
 1. Model weighted Error: 0.7972420371508788

 2. Model Mean Absolute Error:0.8031554324074844
 2. Model weighted Error: 0.8068170841677031

 3. Model Mean Absolute Error:0.7828916062898694
 3. Model weighted Error: 0.7832461930449502

 4. Model Mean Absolute Error:0.8231354739625668
 4. Model weighted Error: 0.8242404553275273

 5. Model Mean Absolute Error:0.792030181878234
 5. Model weighted Error: 0.7936509804061189

 6. Model Mean Absolute Error:0.773159008669674
 6. Model weighted Error: 0.7733065989384583

 7. Model Mean Absolute Error:0.801895155484714
 7. Model weighted Error: 0.8019403016918912

 8. Model Mean Absolute Error:0.807572358892595
 8. Model weighted Error: 0.8081944827902608

 9. Model Mean Absolute Error:0.8690682306211903
 9. Model weighted Error: 0.8704877915514165

 10. Model Mean Absolute Error:0.8338952607303363
 10. Model weighted Error: 0.8355444587656525

